In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [ ]:
def preprocess_image(image_path, sigmas=[1, 2, 3]):
    # Charger l'image en niveaux de gris
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 1. Suppression du bruit (Noise Removal) - Flou Gaussien à différentes échelles (scale space)
    blurred_images = [cv2.GaussianBlur(image, (0, 0), sigma) for sigma in sigmas]

    # 2. Détection des contours (Edges Detection) - Utilisation de Canny à différentes échelles
    edges_at_scales = [cv2.Canny(blurred_image, 100, 200) for blurred_image in blurred_images]

    # 3. Top-Hat et Black-Hat (Morphological Operations)
    kernel = np.ones((5, 5), np.uint8)
    top_hat = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)
    black_hat = cv2.morphologyEx(image, cv2.MORPH_BLACKHAT, kernel)

    # 4. Binarisation avec OTSU
    _, otsu_thresholded = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 5. Binarisation avec Adaptive Threshold
    adaptive_thresholded = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # 6. Normalisation (Normalization) - Ajustement de la luminosité et du contraste
    normalized_image = cv2.equalizeHist(image)

    # Affichage des résultats avec une grille plus grande (4x3)
    plt.figure(figsize=(15, 12))
    
    # Affichage de l'image originale
    plt.subplot(4, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title("Image Originale")
    plt.axis('off')
    
    # Affichage des images floues à différentes échelles
    for i, blurred_image in enumerate(blurred_images):
        plt.subplot(4, 3, i + 2)
        plt.imshow(blurred_image, cmap='gray')
        plt.title(f'Flou Gaussien (Sigma={sigmas[i]})')
        plt.axis('off')
    
    # Affichage des contours détectés
    for i, edges in enumerate(edges_at_scales):
        plt.subplot(4, 3, len(blurred_images) + i + 2)
        plt.imshow(edges, cmap='gray')
        plt.title(f'Contours (Sigma={sigmas[i]})')
        plt.axis('off')
    
    # Affichage des résultats Top-Hat et Black-Hat
    plt.subplot(4, 3, len(blurred_images) + len(edges_at_scales) + 2)
    plt.imshow(top_hat, cmap='gray')
    plt.title("Top-Hat")
    plt.axis('off')

    plt.subplot(4, 3, len(blurred_images) + len(edges_at_scales) + 3)
    plt.imshow(black_hat, cmap='gray')
    plt.title("Black-Hat")
    plt.axis('off')

    # Affichage de la binarisation avec OTSU
    plt.subplot(4, 3, len(blurred_images) + len(edges_at_scales) + 4)
    plt.imshow(otsu_thresholded, cmap='gray')
    plt.title("Binarisation OTSU")
    plt.axis('off')

    # Affichage de la binarisation avec Adaptive Threshold
    plt.subplot(4, 3, len(blurred_images) + len(edges_at_scales) + 5)
    plt.imshow(adaptive_thresholded, cmap='gray')
    plt.title("Binarisation Adaptive")
    plt.axis('off')
    
    plt.show()

    return normalized_image, edges_at_scales, top_hat, black_hat, otsu_thresholded, adaptive_thresholded



In [ ]:

# Exemple d'utilisation
image_path = r"C:\Users\hssin\Downloads\dataset\train\Bulletin_de_soin\0706--8979887--20230705_page_0.jpg"
normalized_image, edges_at_scales, top_hat, black_hat, otsu_thresholded, adaptive_thresholded = preprocess_image(image_path)